In [ ]:
import pandas as pd
from pathlib import Path
import socket
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pyreadr
import pickle

In [ ]:
#paths
fig_path = Path("../figures")
df_path = Path("../dataframes")

#read in UK wide dfs
duration_df = pd.read_csv(df_path / "duration_df.csv")
tha_df = pd.read_csv(df_path / "tha_df.csv")
mhwt_df = pd.read_csv(df_path / "mhwt_df.csv")
severity_df = pd.read_csv(df_path / "severity_df.csv")
mseverity_df = pd.read_csv(df_path / "mseverity_df.csv")
peakvalue_df = pd.read_csv(df_path / "peakvalue_df.csv")

#read in city dfs 
with open("../dataframes/duration_dataframes_dict.pkl", "rb") as file:
    duration_dataframes = pickle.load(file)
with open("../dataframes/th_mhwt_dataframes_dict.pkl", "rb") as file:
    th_mhwt_dataframes = pickle.load(file)
with open("../dataframes/levxally_dataframes_dict.pkl", "rb") as file:
    severity_dataframes = pickle.load(file)
    
#read in city coord df and get wanted cities
result = pyreadr.read_r('/data/users/laura.owen/extremes/heatwaves/HadUKGrid/dur-clim/coords/UK_top30_cities.Rda')
city_df = result['city_df']
city_names_in_dataframes = set(duration_dataframes.keys())
city_df = city_df[city_df['city'].isin(city_names_in_dataframes)].copy()
#remove city=Islington as its the same as London
city_df = city_df[~((city_df['city'] == 'Islington'))]

#edit duration df so that its the no. of days per year/season (need to divide by 100,000)
duration_cols = [col for col in duration_df.columns if col.startswith("duration")]
duration_df[duration_cols] = duration_df[duration_cols] / 100000
for city, df in duration_dataframes.items():
    df["nduration"] /= 100000

RPy = [0.5, 2, 5, 10, 20, 50, 100, 200, 500, 1000]